In [1]:
# 다음에 해볼 것
# 1. 전처리 더 하기
# 2. lemmatize 해보기
# 3. 연도별 잘라서 관측
# 4. 저자 EDA (소속, 나라)
# 5. 최적 토픽 수
# 6. 하이퍼 파라미터 바꿔보기 (알파, 베타)

In [2]:
import gensim
import pickle
import pandas as pd


In [3]:
df = pd.read_pickle("../files/final_df.pickle")

In [4]:
# load corpus, dictionary

with open('../files/lda/corpus.pickle', 'rb') as f:
    corpus = pickle.load(f) # 단 한줄씩 읽어옴

with open('../files/lda/dictionary.pickle', 'rb') as f:
    dictionary = pickle.load(f) # 단 한줄씩 읽어옴


In [7]:
model = gensim.models.ldamodel.LdaModel.load("../files/lda/lda.model")

In [43]:
topics = model.print_topics(num_words=100)

In [44]:
for t in topics:
    print(t)

# 일단 작성
not_decided_del_words = ["P.", "A.", "B."]

del_words = ["%", ">", "<", "[", "]", "P", "p", "+/-", "mm", "found", "=", "CI"]

(0, '0.021*"treatment" + 0.017*"periodontal" + 0.017*"probing" + 0.016*"clinical" + 0.015*"group" + 0.015*"patients" + 0.013*"sites" + 0.013*"therapy" + 0.012*"study" + 0.012*"mm" + 0.011*"months" + 0.011*"significant" + 0.010*"depth" + 0.010*"plaque" + 0.010*"attachment" + 0.010*"%" + 0.010*"control" + 0.010*"groups" + 0.010*"baseline" + 0.009*"pocket" + 0.008*"scaling" + 0.008*"root" + 0.007*"bleeding" + 0.007*"gingival" + 0.007*"reduction" + 0.006*"planing" + 0.006*"test" + 0.006*"parameters" + 0.006*"index" + 0.006*"pockets" + 0.006*"periodontitis" + 0.006*"level" + 0.006*"compared" + 0.005*"<" + 0.005*"mean" + 0.005*"weeks" + 0.005*"PD" + 0.005*"P" + 0.005*"following" + 0.005*"showed" + 0.005*"effect" + 0.004*"treated" + 0.004*"subgingival" + 0.004*"period" + 0.004*"SRP" + 0.004*"results" + 0.004*"days" + 0.004*"received" + 0.004*"differences" + 0.004*"greater" + 0.004*"=" + 0.003*"p" + 0.003*"effects" + 0.003*"initial" + 0.003*"time" + 0.003*"subjects" + 0.003*"using" + 0.003*"de

In [21]:
!pip install tqdm

In [25]:
from tqdm.notebook import tqdm

In [31]:
topic_1_list = []
check_topic = model[corpus]

for i, topic_dist in tqdm(enumerate(check_topic)):
    for idx, dist in topic_dist:
        if idx == 1 and dist >= 0.5:
            topic_1_list.append(i)

In [29]:
# for문 예시
import time

for i in tqdm(range(10000)):
    time.sleep(0.001)

In [40]:
df.iloc[topic_1_list, 2][:10].values

array(['Doxycycline hyclate-loaded in situ forming gels composed from bleached shellac,\nEthocel, and Eudragit RS for periodontal pocket delivery.',
       'Decreased temperature increases the expression of a disordered bacterial late\nembryogenesis abundant (LEA) protein that enhances natural transformation.',
       'Mammalian-like type II glutaminyl cyclases in Porphyromonas gingivalis and other \noral pathogenic bacteria as targets for treatment of periodontitis.',
       'Structural and kinetic characterization of Porphyromonas gingivalis glutaminyl\ncyclase.',
       'The association of apical periodontitis and type 2 diabetes mellitus: A large\nhospital network cross-sectional case-controlled study.',
       'Complementation in trans of Porphyromonas gingivalis Lipopolysaccharide\nBiosynthetic Mutants Demonstrates Lipopolysaccharide Exchange.',
       'Synergistic effects of D-arginine, D-methionine and D-histidine against\nPorphyromonas gingivalis biofilms.',
       'Porphyromo

In [46]:
# 연도별 잘라서 관측
df.head()


for i, topic_list in enumerate(model[corpus]):
    if i == 5:
        break
    print(topic_list)
# 각 문서당 토픽의 분포를 알고 있음
# 1. 10년 단위로 문서를 묶는다
# 2. 년도에서 토픽의 분포를 전부 더해줌
# 3. 가장 마지막에 문서의 갯수로 나눠줌


[(1, 0.15113613), (2, 0.058725297), (4, 0.032367926), (7, 0.75353163)]
[(7, 0.047067277), (8, 0.022609705), (9, 0.9250188)]
[(1, 0.69058776), (3, 0.07118194), (4, 0.05214687), (6, 0.018419385), (7, 0.14275612), (9, 0.02108088)]
[(0, 0.5703713), (9, 0.42525983)]
[(3, 0.082840964), (5, 0.2271941), (6, 0.55434394), (9, 0.13042973)]


In [59]:
df.reset_index(drop=True, inplace=True)

In [61]:
# 1980년대 논문으로 테스트
test = df[(df.year >= '1980') & (df.year < '1990')]
test.head()

,journal_name,year,title,author,author_info,abstract,pos_tag_abstract
29999,Z Stomatol,1989,[Oral manifestations of HIV infection].,[Article in German],"Puelacher W, Zangerle R, Kulmer S, Waldhart E,...",Skin and mucous membranes including the oral m...,"[Skin, mucous, membranes, including, oral, muc..."
30000,Zh Mikrobiol Epidemiol Immunobiol,1989,[The tentative identification of actinomycetes...,[Article in Russian],"Mel'nikov VG, Oleĭnik II.",In this work the results obtained in the study...,"[work, results, obtained, study, morphology, A..."
30001,Aust Dent J,1989,Differential diagnosis of severe periodontal l...,"Hirsch RS, Clarke NG.",Comment in\n Aust Dent J. 1990 Apr;35(2):19...,"In current clinical practice, a differential d...","[current, clinical, practice, differential, di..."
30002,Scand J Dent Res,1989,"Gingival fluid, beta 2-microglobulin and prote...","Syrjänen SM(1), Alakuijala P, Markkanen SO, Ma...","Author information: \n(1)Faculty of Dentistry,...","beta 2-microglobulin (beta 2-m), lysozyme and ...","[beta, lysozyme, protein, concentrations, ging..."
30003,J Periodontol,1989,A comparison of freeze-dried bone allograft an...,"Rummelhart JM(1), Mellonig JT, Gray JL, Towle HJ.","Author information: \n(1)Naval Dental Clinic, ...",This study was conducted to clinically compare...,"[study, conducted, compare, freeze-dried, bone..."


In [71]:
# 1980년대 논문만 corpus에서 가져옴

index_1980 = list(test.index)
# corpus_1980 = corpus[0,1,2,3]
corpus_1980 = corpus[29999:30726]

In [76]:
dist_dict = {i:0 for i, value in enumerate(range(10))}

# dist_dict
for topic_dist in model[corpus_1980]:
    for key, value in topic_dist:
        dist_dict[key] += value

In [79]:
for key, value in dist_dict.items():
    dist_dict[key] = dist_dict[key] / len(corpus_1980)

In [80]:
dist_dict

{0: 0.14839869738009437,
 1: 0.08535079164699519,
 2: 0.07321674554535743,
 3: 0.07672238255757455,
 4: 0.09521789828051481,
 5: 0.03594253302655571,
 6: 0.07649964624003826,
 7: 0.1440153460157971,
 8: 0.15973071777934345,
 9: 0.09511739485467972}

In [82]:
# 합이 1에 가깝게 잘 나옴
sum(dist_dict.values())

0.9902121533269507